In [1]:
import requests
import json



import matplotlib.pyplot as plt
import seaborn as sns

from pandasdmx import Request
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd
import numpy as np
import geopandas as gpd

import time

import csv

import sys, os
from sqlalchemy import create_engine

import pymssql
import sqlalchemy as db

from functools import reduce

import folium # map rendering library
from sklearn.neighbors import NearestNeighbors



/home/holborn/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [ ]:
#intialise Foursquare API credentials
url = 'https://api.foursquare.com/v2/venues/explore'


CLIENT_ID = 'ZY2UKTXC3KLI2DSJYFZPC3CPXMU4SGKILQOKDZK5FAMGXB2C' # your Foursquare ID
CLIENT_SECRET = '0D0TWC2MD2NDOHUUXRAYAS1DRQRAA2E14Z33QU01YADTE0O1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


# #intialise Foursquare API credentials
# CLIENT_ID = 'EKC0OWGJC1SY1AE1UHB4PUPH2JGARZTQK1U5C1USTUNA43JF' # your Foursquare ID
# CLIENT_SECRET = 'CHH0EPUEN2PH4WBDV4XHTDT5NWTUZ2SYVIQALUOZXWYNABRO' # your Foursquare Secret

In [ ]:

#Use the function to extract the category from the dataframe (because the column name could be either 'categories' or 'venue.categories')
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


#define a function to return all the venues for a given neighborhood

#pass the neighborhood (nb), the latitude (la), the radius and the limit 
def get_venues(nb, la, lo, radius, limit):
    
    #print(nb)
    
    #form the request url and request only the items (which are the venues)
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, la, lo, VERSION, limit)
    items = requests.get(url).json()["response"]['groups'][0]['items']
    
    #include exception handling where Foursquare request fails for a particular Neighbourhood
    if items == []:
        print(" -- Foursquare request for ",nb,"returned no results --")
        return None
    else:
        # flatten JSON, filter for only wanted columns then use the get_category_type funtion to replace the category list with just the category.
        venues1 = json_normalize(items)
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        venues1 =venues1.loc[:, filtered_columns]
        venues1['venue.categories'] = venues1.apply(get_category_type, axis=1)

        #define a new dataframe with the Neighbourhood information
        venues2 = pd.DataFrame(columns=['Suburb','Suburb Latitude', 
                      'Suburb Longitude']) 

        #for each of the venues returned, add the venue dataframe to the (empty) neighbourhood dataframe and fill all rows in the the neighbourhood columns with the neighborhood name, the neighbourhood latitude and longitude
        i=0
        for row in venues1:
            venues2[row]=venues1[row]
            venues2['Suburb']=nb
            venues2['Suburb Latitude']=la
            venues2['Suburb Longitude']=lo
            i=i+1

        #rename the columns
        venues2.rename(index=str,columns={"venue.name":"Venue","venue.categories":"Venue Category","venue.location.lat":"Venue Latitude","venue.location.lng":"Venue Longitude"}, inplace=True)
    
    return venues2

In [ ]:
# #Iterate thorough all the suburbs of the dataframe df, adding each set of venues to the df_suburb_venues dataframe
# #df=melb_postcodes
# df=melb_postcodes
# df_suburb_venues=pd.DataFrame()
# radius = 500
# limit = 10000
# print("Getting venue recommendations, please wait......")
# for i, row in enumerate(df['suburb']):
#         df_suburb_venues=df_suburb_venues.append(get_venues(df.iloc[i]['suburb'], df.iloc[i]['lat'], df.iloc[i]['lon'], radius, limit),ignore_index=True)
# print("...... Complete")